In [ ]:
import teehr
import teehr.example_data.v0_3_test_study as test_study_data
from pathlib import Path
import shutil
import geopandas as gpd
import pandas as pd

import pyspark

from pyspark.sql import SparkSession
from pyspark import SparkConf

# Tell Bokeh to output plots in the notebook
from bokeh.io import output_notebook
output_notebook()


In [ ]:
# from delta import *

# conf = (
#     SparkConf()
#     .setAppName("TEEHR")
#     .setMaster("local[*]")
#     .set("spark.sql.sources.partitionOverwriteMode", "dynamic")
#     .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
#     .set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider")
#     .set("spark.sql.execution.arrow.pyspark.enabled", "true")
#     .set("spark.sql.session.timeZone", "UTC")
#     .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
#     .set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
#     .set("spark.sql.package", "io.delta:delta-core_2.12:2.1.0")
# )

# builder = pyspark.sql.SparkSession.builder.config(conf=conf)
# spark = configure_spark_with_delta_pip(builder).getOrCreate()


In [ ]:
# conf = (
#     SparkConf()
#     .setAppName("TEEHR")
#     .setMaster("local[*]")
#     .set("spark.sql.sources.partitionOverwriteMode", "dynamic")
#     .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
#     .set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider")
#     .set("spark.sql.execution.arrow.pyspark.enabled", "true")
#     .set("spark.sql.session.timeZone", "UTC")
#     .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
#     .set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
#     .set("spark.sql.package", "io.delta:delta-core_2.12:2.1.0")
#     .set("spark.sql.warehouse.dir", warehouse_location) \
# )
# spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [ ]:
# Define the directory where the Evaluation will be created
test_eval_dir = Path(Path().home(), "temp", "02_delta_tables")
shutil.rmtree(test_eval_dir, ignore_errors=True)

# Create an Evaluation object and create the directory
ev = teehr.Evaluation(
    dir_path=test_eval_dir,
    create_dir=True
)

# Clone the template
ev.clone_template()

In [ ]:
location_data_path = Path(test_eval_dir, "gages.geojson")
test_study_data.fetch_file("gages.geojson", location_data_path)

In [ ]:
location_data_path = Path(test_eval_dir, "gages.geojson")
gdf = gpd.read_file(location_data_path)
gdf

In [ ]:
ev.locations._create_delta_table()

In [ ]:
ev.locations.to_sdf().show()

In [ ]:
ev.locations.load_spatial(location_data_path)

In [ ]:
ev.locations.to_geopandas()

In [ ]:
primary_timeseries_path = Path(test_eval_dir, "test_short_obs.csv")
test_study_data.fetch_file("test_short_obs.csv", primary_timeseries_path)

In [ ]:
primary_timeseries_df = pd.read_csv(primary_timeseries_path)
primary_timeseries_df.head()

In [ ]:
ev.units.to_pandas()

In [ ]:
ev.variables.to_pandas()

In [ ]:
ev.configurations.to_pandas()

In [ ]:
ev.configurations.add(
    teehr.Configuration(
        name="usgs_observations",
        type="primary",
        description="Test Observed Configuration",
    )
)

In [ ]:
ev.configurations.to_pandas()

In [ ]:
ev.primary_timeseries._create_delta_table()

In [ ]:
# Load the timeseries data and map over the fields and set constants
ev.primary_timeseries.load_csv(
    in_path=primary_timeseries_path,
    field_mapping={
        "reference_time": "reference_time",
        "value_time": "value_time",
        "configuration": "configuration_name",
        "measurement_unit": "unit_name",
        "variable_name": "variable_name",
        "value": "value",
        "location_id": "location_id"
    },
    constant_field_values={
        "unit_name": "m^3/s",
        "variable_name": "streamflow_hourly_inst",
        "configuration_name": "usgs_observations"
    },
    # location_id_prefix="usgs",
    # write_mode="overwrite"
)

In [ ]:
ev.primary_timeseries.to_sdf().count()